In [1]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()

0

In [3]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [4]:
processed_data_dir_path = "/workspace/data_processed"

In [5]:
import glob 
data_open_slr_paths = glob.glob("/workspace/data/external/*")

In [6]:
data_open_slr_paths

['/workspace/data/external/13',
 '/workspace/data/external/2',
 '/workspace/data/external/7',
 '/workspace/data/external/3',
 '/workspace/data/external/4',
 '/workspace/data/external/5',
 '/workspace/data/external/8',
 '/workspace/data/external/6',
 '/workspace/data/external/10',
 '/workspace/data/external/9',
 '/workspace/data/external/12',
 '/workspace/data/external/1',
 '/workspace/data/external/11']

In [7]:
# Creating Processor
data_1 = '/workspace/data/external/13'

In [8]:
transcript_json = data_1 + "/" +  "data.json"
print(transcript_json)
data = pd.read_json(transcript_json)

/workspace/data/external/13/data.json


In [9]:
data['audioFilename']

0        9_Regional-Guwahati-Nepali-1515-2019331153347.wav
1        9_Regional-Guwahati-Nepali-1515-2019910155032.wav
2         9_Regional-Guwahati-Nepali-1515-201999152725.wav
3        9_Regional-Guwahati-Nepali-1515-2019731153014.wav
4        9_Regional-Guwahati-Nepali-1515-2020115152955.wav
                               ...                        
16437    3_2_Regional-Kurseong-Nepali-1850-201822619214...
16438    3_2_Regional-Kurseong-Nepali-1850-201812519354...
16439    3_2_Regional-Kurseong-Nepali-1850-201910262114...
16440    3_2_Regional-Kurseong-Nepali-1850-201911161992...
16441    3_2_Regional-Kurseong-Nepali-1850-201911220226...
Name: audioFilename, Length: 16442, dtype: object

In [10]:
for i in range(len(data)):
    file = data_1  + "/" + data['audioFilename'][i]
    data['audioFilename'][i] = file

In [11]:
data['audioFilename']

0        /workspace/data/external/13/9_Regional-Guwahat...
1        /workspace/data/external/13/9_Regional-Guwahat...
2        /workspace/data/external/13/9_Regional-Guwahat...
3        /workspace/data/external/13/9_Regional-Guwahat...
4        /workspace/data/external/13/9_Regional-Guwahat...
                               ...                        
16437    /workspace/data/external/13/3_2_Regional-Kurse...
16438    /workspace/data/external/13/3_2_Regional-Kurse...
16439    /workspace/data/external/13/3_2_Regional-Kurse...
16440    /workspace/data/external/13/3_2_Regional-Kurse...
16441    /workspace/data/external/13/3_2_Regional-Kurse...
Name: audioFilename, Length: 16442, dtype: object

In [12]:
data.rename(columns = {'text':'sentence'}, inplace = True)
data.rename(columns = {'audioFilename':'path'}, inplace = True)

In [13]:
import os
rows_to_be_deleted = []
def check_0_audio_remove(df):
    for i in range(len(df)):
        if(os.path.getsize(df['path'][i]) == 0):
            rows_to_be_deleted.append(i)


check_0_audio_remove(data)

In [14]:
data.drop(data.index[rows_to_be_deleted], inplace=True)


In [15]:
data = data.reset_index()


In [16]:
len(data)

16348

In [17]:
train = data[10000:]
len(train)

6348

In [18]:
common_voice_train_1 = Dataset.from_pandas(train[0:3000])
common_voice_train_2 = Dataset.from_pandas(train[3000:])

print(len(common_voice_train_1))
print(len(common_voice_train_2))

3000
3348


In [19]:
print(common_voice_train_1['sentence'][4])
print(common_voice_train_2['sentence'][4])

तर आफ्नो गति लिइ रहेछ
काकडा मटर प्याज आलु र चमर हुन्


In [20]:
common_voice_train_1 = common_voice_train_1.remove_columns(["collectionSource","snr",  "duration", "gender"])
common_voice_train_2 = common_voice_train_2.remove_columns(["collectionSource","snr",  "duration", "gender"])

In [21]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [22]:
common_voice_train_1 = common_voice_train_1.map(remove_special_characters)
common_voice_train_2 = common_voice_train_2.map(remove_special_characters)

In [23]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [24]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)

In [25]:
common_voice_train_2 = common_voice_train_2.map(speech_file_to_array_fn, remove_columns=common_voice_train_2.column_names,num_proc=4)

In [26]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train_1)-1)

ipd.Audio(data=np.asarray(common_voice_train_1[rand_int]["speech"]), autoplay=True, rate=16000)

In [27]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [28]:
tokenizer = Wav2Vec2CTCTokenizer("/workspace/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [29]:
len(processor.tokenizer)

62

In [30]:
common_voice_train_1 = common_voice_train_1.map(prepare_dataset, remove_columns=common_voice_train_1.column_names, batch_size=8, num_proc=4, batched=True)

In [31]:
common_voice_train_2 = common_voice_train_2.map(prepare_dataset, remove_columns=common_voice_train_2.column_names, batch_size=8, num_proc=4, batched=True)

In [32]:
common_voice_train_2

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 3348
})

In [33]:
common_voice_train = concatenate_datasets([common_voice_train_1, common_voice_train_2])

In [34]:
processed_data_dir_path_ = processed_data_dir_path + "/" + str(39)

In [35]:
processed_data_dir_path_

'/workspace/data_processed/39'

In [36]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 6348
})

In [37]:
common_voice_train.save_to_disk(processed_data_dir_path_)